In [1]:
# Install eflomal
!git clone https://github.com/robertostling/eflomal && cd eflomal


Cloning into 'eflomal'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 109.65 KiB | 6.45 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [2]:
%cd eflomal

/content/eflomal


In [3]:
!make

cc -Ofast -march=native -Wall --std=gnu99 -Wno-unused-function -g -fopenmp -c eflomal.c
eflomal.c: In function ‘text_alignment_load_priors’:
eflomal.c:858:9: warning: dereferencing type-punned pointer will break strict-aliasing rules [-Wstrict-aliasing]
         map_token_u32_add(ta->source_prior + e, f, *((uint32_t*)&alpha));
         ^~~~~~~~~~~~~~~~~
In file included from eflomal.c:53:0:
random.c: In function ‘random_system_state’:
random.c:73:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
     fread(state, sizeof(*state), 1, file);
     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
cc -lm -lrt -lgomp -fopenmp  eflomal.o   -o eflomal


In [4]:
!sudo make install

install -t /usr/local/bin eflomal


In [5]:
!python3 setup.py install

Compiling python/eflomal/eflomal.pyx because it changed.
[1/1] Cythonizing python/eflomal/eflomal.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/eflomal/python/eflomal/eflomal.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running install
running bdist_egg
running egg_info
creating eflomal.egg-info
writing eflomal.egg-info/PKG-INFO
writing dependency_links to eflomal.egg-info/dependency_links.txt
writing requirements to eflomal.egg-info/requires.txt
writing top-level names to eflomal.egg-info/top_level.txt
writing manifest file 'eflomal.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'eflomal.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'eflomal' extension
creating build
creating build/temp.linux-x86_64-3.8
creating b

In [33]:
# Open the Parlamint file
import pandas as pd

df = pd.read_csv("/content/Parlamint-SI-bigger-sample-translated.csv", sep="\t", index_col = 0)
df.head()

,file,sentence_id,text,proper_nouns,length,translation
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,[],7,I am starting a continuation of the 12th meeti...
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","[{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19...",89,I am informed that today's meeting cannot be a...
2,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.2,Vsem prisotnim še enkrat lep pozdrav !,[],7,I'm here to say hello again!
3,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg4.1,"Prehajamo na 2. , 3. , 4. IN 5. TOČKO DNEVNEGA...",[],70,"Moving to 2, 3, 4. AND 5. DETECTION OF THE DAY..."
4,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg5.1,Predloge zakonov je v obravnavo Državnemu zbo...,[],10,Proposals for laws were submitted by the Gover...


In [7]:
df.describe(include="all")

,file,sentence_id,text,proper_nouns,length,translation
count,115993,115993,115993,115993,115993.000000,115993
unique,40,115993,102752,7333,NaN,101985
top,ParlaMint-SI_2016-11-23-SDZ7-Redna-24.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Hvala lepa .,[],NaN,Thank you very much.
freq,6185,1,3029,105521,NaN,3169
mean,NaN,NaN,NaN,NaN,21.526661,NaN
std,NaN,NaN,NaN,NaN,16.683888,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN
25%,NaN,NaN,NaN,NaN,9.000000,NaN
50%,NaN,NaN,NaN,NaN,18.000000,NaN
75%,NaN,NaN,NaN,NaN,30.000000,NaN


In [9]:
# The English translations did not preserve what we did in Slovene where we put spaces before all punctuation marks.
# We will do this now.

translation_list = df.translation.to_list()

import re

translation_added_spaces = []

for translation in translation_list:
  # Add spaces around each punctuation
  translation = re.sub('([.,!?:;()])', r' \1 ', translation)
  # Remove duplicated spaces
  translation = re.sub('\s{2,}', ' ', translation)
  translation_added_spaces.append(translation)

df["translation_added_spaces"] = translation_added_spaces

df.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,[],7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","[{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...
2,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.2,Vsem prisotnim še enkrat lep pozdrav !,[],7,I'm here to say hello again!,I'm here to say hello again !
3,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg4.1,"Prehajamo na 2. , 3. , 4. IN 5. TOČKO DNEVNEGA...",[],70,"Moving to 2, 3, 4. AND 5. DETECTION OF THE DAY...","Moving to 2 , 3 , 4 . AND 5 . DETECTION OF THE..."
4,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg5.1,Predloge zakonov je v obravnavo Državnemu zbo...,[],10,Proposals for laws were submitted by the Gover...,Proposals for laws were submitted by the Gover...


In [10]:
# Create files for all texts and all translations

Slovene_sentences = open("Slovene_sentences.txt", "w")
English_sentences = open("English_sentences.txt", "w")

for i in df.text.to_list():
	Slovene_sentences.write(i)
	Slovene_sentences.write("\n")

for i in df.translation_added_spaces.to_list():
	English_sentences.write(i)
	English_sentences.write("\n")

Slovene_sentences.close()
English_sentences.close()

In [ ]:
# When used with the -s and -t options for separate source/target files, the align.py interface expects one sentence
# per line with space-separated tokens, similar to most word alignment software.

In [11]:
# Align sentences with eflomal
!python3 align.py -s "/content/eflomal/Slovene_sentences.txt" -t "/content/eflomal/English_sentences.txt" --model 3 -f sl-en.fwd -r sl-en.rev

Eflomal needed 1 minute to align 11,750 sentences and 238,959 words (3 files). For 115,993 sentences (40 files), it needed 5 minutes.

Let's create a list of alignments from the output file - I think the rev file is more useful for us, because we are interested with which word the Slovene word is aligned, to change that word. E.g., in pair "Hvala lepa predsedujoči"	- "Thank you very much, President.", the forward alignment is "0-0 0-1 1-2 1-3 2-4" and the reverse alignment is "0-0 1-3 2-4". For instance, if we want to know to which word "lepa" (index 1) is aligned, it is more useful to have "much" as the answer, than 2 answers ("very", "much").

The analysis showed that in the fwd file some words are aligned to multiple words while this is not the case in the reverse file.

Another thing with eflomal that we need to be aware of is that it doesn't include "null" results into the fwd and rev files - e.g. a line in a rev file was empty (it did not manage to align "Je ! " with "Yes!".

In [12]:
aligns_file = open("/content/eflomal/sl-en.rev", "r")

aligns_list = aligns_file.readlines()

aligns_list = [i.replace("\n", "") for i in aligns_list]

aligns_list = [i.split(" ") for i in aligns_list]

print(len(aligns_list))
print(aligns_list[:1])

115993
[['0-2', '1-4', '2-7', '3-8', '4-11', '5-12', '6-13']]


In [13]:
final_aligns = []

for i in aligns_list:
  current_line = {}
  
  try:
    for element in i:
      new_list = element.split("-")

      a = int(new_list[0])
      b = int(new_list[1])
      current_line[a] = b
  
    # Check whether the number of pairs in the list is the same as number of items
    if len(i) != len(list(current_line.items())):
      print("Not okay:")
      print(i)
      print(current_line)

    final_aligns.append(current_line)
  
  except:
    print("error")
    print(aligns_list.index(i))
    print(i)
    final_aligns.append("Error")

final_aligns[4]


error
15212
['']


{0: 0, 1: 2, 2: 3, 4: 4, 5: 10, 6: 11, 7: 5, 8: 7, 9: 14}

In [14]:
len(final_aligns)

115993

In [15]:
# Add a to the df

df["alignments"] = final_aligns

df.head(2)

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,[],7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...,"{0: 2, 1: 4, 2: 7, 3: 8, 4: 11, 5: 12, 6: 13}"
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","[{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:..."


In [16]:
# Change strings in the column proper_nouns into dictionaries
import ast


df["proper_nouns"] = df.proper_nouns.astype("str")
df["proper_nouns"] = df.proper_nouns.apply(lambda x: ast.literal_eval(x))

# Remove the list brackets
df["proper_nouns"] = df.proper_nouns.str[0]

df.head(2)

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,NaN,7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...,"{0: 2, 1: 4, 2: 7, 3: 8, 4: 11, 5: 12, 6: 13}"
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19:...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:..."


In [17]:
# Change nan values
df = df.fillna(0)
df.head(2)

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,0,7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...,"{0: 2, 1: 4, 2: 7, 3: 8, 4: 11, 5: 12, 6: 13}"
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19:...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:..."


In [18]:
# Substitute words in the translation based on alignments
intermediate_list = list(zip(df["translation_added_spaces"], df["proper_nouns"], df["alignments"]))

new_translations = []
substituted_all_info = []
substituted_only = []

# Add information whether an error occurred
error_list = []

for i in intermediate_list:
  current_substituted_list = []
  current_substituted_only = []
  current_error = "No"

# If no proper names were detected, do not change the translation
  if i[1] == 0:
    new_translations.append(i[0])
  
  else:
    current_translation = i[0]

    # Substitute the word with the Slovene lemma based on the index - loop through the proper nouns to be changed
    for word_index in list(i[1].keys()):
      try:
        # split the translation into list of words
        word_list = current_translation.split()

        # Get index of the substituted word
        substituted_word_index = i[2][word_index]

        # Get the lemma to substitute the word with
        correct_lemma = i[1][word_index][1]

        # If the substitute word and lemma are not the same, get substituted word and its match
        if word_list[substituted_word_index] != correct_lemma:
          current_substituted_list.append((word_list[substituted_word_index], correct_lemma))
          current_substituted_only.append((word_list[substituted_word_index], correct_lemma))

          # Substitute the word in the word list
          word_list[substituted_word_index] = correct_lemma
          
        else:
        # Add information that substitution was not performed
          current_substituted_list.append(f"No substitution: {word_list[substituted_word_index], correct_lemma}")
        
        # Change the translation by merging the words back into a string
        current_translation = " ".join(word_list)

      except:
        print(f"Issue: index {word_index}: {i[1][word_index]}")
        current_error = f"Issue: index {word_index}: {i[1][word_index]}"

    # After the loop through proper nouns, save the new translation
    new_translations.append(current_translation)
  
  # Add information on what was substituted
  substituted_all_info.append(current_substituted_list)
  substituted_only.append(current_substituted_only)
  error_list.append(current_error)

Issue: index 17: ['Trček', 'Trček']
Issue: index 20: ['Šircelj', 'Šircelj']
Issue: index 14: ['Pavšič', 'Pavšič']
Issue: index 10: ['Liste', 'lista']
Issue: index 1: ['Trček', 'Trček']
Issue: index 4: ['Marjana', 'Marjan']
Issue: index 5: ['Šarca', 'Šarec']
Issue: index 12: ['Pavšič', 'Pavšič']
Issue: index 1: ['Matić', 'Matić']
Issue: index 4: ['Mira', 'Miro']
Issue: index 5: ['Cerarja', 'Cerar']
Issue: index 16: ['Židanu', 'Židan']
Issue: index 3: ['Samo', 'Samo']
Issue: index 0: ['Klemenčiču', 'Klemenčič']
Issue: index 27: ['Škrlec', 'Škrlec']
Issue: index 80: ['Mesec', 'Mesec']
Issue: index 2: ['Počivalšek', 'Počivalšek']
Issue: index 6: ['Ali', 'ali']
Issue: index 4: ['Evroskupina', 'Evroskupina']
Issue: index 9: ['Izvolite', 'izvoliti']
Issue: index 50: ['Lep', 'lep']
Issue: index 1: ['Pojbič', 'Pojbič']
Issue: index 1: ['Trček', 'Trček']
Issue: index 5: ['Cerarja', 'Cerar']
Issue: index 8: ['Sicer', 'sicer']
Issue: index 15: ['Židanu', 'Židan']
Issue: index 3: ['Mira', 'Miro']
I

In [19]:
# Add to the df

df["new_translations"] = new_translations
df["substitution_info"] = substituted_all_info
df["substituted_words"] = substituted_only
df["errors"] = error_list

df.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments,new_translations,substitution_info,substituted_words,errors
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,0,7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...,"{0: 2, 1: 4, 2: 7, 3: 8, 4: 11, 5: 12, 6: 13}",I am starting a continuation of the 12th meeti...,[],[],No
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19:...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:...",I am informed that today's meeting cannot be a...,"[No substitution: ('Eva', 'Eva'), No substitut...","[(Vrtovac, Vrtovec), (Serva, Sluga), (Suzanne,...",No
2,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.2,Vsem prisotnim še enkrat lep pozdrav !,0,7,I'm here to say hello again!,I'm here to say hello again !,"{0: 2, 1: 4, 3: 5, 4: 4, 5: 4, 6: 6}",I'm here to say hello again !,[],[],No
3,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg4.1,"Prehajamo na 2. , 3. , 4. IN 5. TOČKO DNEVNEGA...",0,70,"Moving to 2, 3, 4. AND 5. DETECTION OF THE DAY...","Moving to 2 , 3 , 4 . AND 5 . DETECTION OF THE...","{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: ...","Moving to 2 , 3 , 4 . AND 5 . DETECTION OF THE...",[],[],No
4,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg5.1,Predloge zakonov je v obravnavo Državnemu zbo...,0,10,Proposals for laws were submitted by the Gover...,Proposals for laws were submitted by the Gover...,"{0: 0, 1: 2, 2: 3, 4: 4, 5: 10, 6: 11, 7: 5, 8...",Proposals for laws were submitted by the Gover...,[],[],No


In [37]:
# See substitutions

df_substituted = df[df["proper_nouns"] != "0"]

df_substituted.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments,new_translations,substitution_info,substituted_words,errors
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19:...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:...",I am informed that today's meeting cannot be a...,"[""No substitution: ('Eva', 'Eva')"", ""No substi...","[('Vrtovac', 'Vrtovec'), ('Serva', 'Sluga'), (...",No
5,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg5.2,Zato vabim gospoda ministra dr. Andreja Berto...,"{5: ['Andreja', 'Andrej'], 6: ['Bertonclja', '...",11,That is why I invite Mr. Minister Andrew Berto...,That is why I invite Mr . Minister Andrew Bert...,"{0: 2, 1: 4, 2: 5, 3: 7, 5: 8, 6: 9, 8: 10, 9:...",That is why I invite Mr . Minister Andrej Bert...,"[('Andrew', 'Andrej'), ('Bertoncl', 'Bertoncel...","[('Andrew', 'Andrej'), ('Bertoncl', 'Bertoncel...",No
79,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg18.1,Pristojno delovno telo je Odbor za finance i...,"{13: ['Robertu', 'Robert'], 14: ['Polnarju', '...",16,The competent working body is the Committee on...,The competent working body is the Committee on...,"{0: 1, 1: 2, 2: 3, 3: 4, 4: 6, 5: 7, 6: 8, 7: ...",The competent working body is the Committee on...,"[""No substitution: ('Robert', 'Robert')"", ""No ...",[],No
111,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg30.2,"Pojasnila je , da se z amandmajema koalicijski...","{20: ['Levice', 'Levica']}",28,She explained that the amendments to the coali...,She explained that the amendments to the coali...,"{0: 1, 3: 2, 5: 3, 6: 4, 7: 7, 8: 8, 9: 9, 10:...",She explained that the amendments to the coali...,"[('Left', 'Levica')]","[('Left', 'Levica')]",No
170,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg48.1,Kot prvi imate predstavitev gospod Jani Predn...,"{5: ['Jani', 'Jani'], 6: ['Prednik', 'Prednik']}",14,"As the first one, you have a presentation by M...","As the first one , you have a presentation by ...","{0: 0, 1: 2, 2: 6, 3: 8, 4: 10, 5: 11, 6: 12, ...","As the first one , you have a presentation by ...","[""No substitution: ('Jani', 'Jani')"", ""No subs...",[],No


In [38]:
df_substituted.shape

(10472, 12)

In [39]:
# Check errors:
print(df_substituted[df_substituted["errors"] != "No"][["text", "translation", "new_translations", "alignments", "errors"]].to_markdown())

|        | text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | translation                                                                                                                                                                                                                                                                                

In [45]:
df_substituted.substituted_words.describe()

count     10472
unique     2246
top          []
freq       4433
Name: substituted_words, dtype: object

In [60]:
print(df_substituted[df_substituted["substituted_words"] != "[]"].substituted_words.value_counts()[:80].to_markdown())

|                                                           |   substituted_words |
|:----------------------------------------------------------|--------------------:|
| [('Mir', 'Miro')]                                         |                 157 |
| [('Slovenians', 'Slovenec')]                              |                 141 |
| [('Left', 'Levica')]                                      |                 112 |
| [('Joseph', 'Jožef')]                                     |                 112 |
| [('Goranak', 'Gorenak')]                                  |                  98 |
| [('Weber', 'Veber')]                                      |                  77 |
| [('Sarca', 'Šarec')]                                      |                  61 |
| [('Levia', 'Levica')]                                     |                  60 |
| [('Fisher', 'Fišer')]                                     |                  57 |
| [('Matthew', 'Matej')]                                    |               

In [41]:
print(df_substituted.substituted_words.value_counts()[:20].to_markdown())

|                              |   substituted_words |
|:-----------------------------|--------------------:|
| []                           |                4433 |
| [('Mir', 'Miro')]            |                 157 |
| [('Slovenians', 'Slovenec')] |                 141 |
| [('Left', 'Levica')]         |                 112 |
| [('Joseph', 'Jožef')]        |                 112 |
| [('Goranak', 'Gorenak')]     |                  98 |
| [('Weber', 'Veber')]         |                  77 |
| [('Sarca', 'Šarec')]         |                  61 |
| [('Levia', 'Levica')]        |                  60 |
| [('Fisher', 'Fišer')]        |                  57 |
| [('Matthew', 'Matej')]       |                  55 |
| [('Trtek', 'Trček')]         |                  53 |
| [('Mira', 'Miro')]           |                  52 |
| [('Jean', 'Žan')]            |                  51 |
| [('Ange', 'Anže')]           |                  48 |
| [('Vrtovac', 'Vrtovec')]     |                  43 |
| [('Serva

In [22]:
# Save the df
df.to_csv("Parlamint-SI-sample-aligned-corrected.csv", sep="\t")

In [35]:
# Open the df
df = pd.read_csv("Parlamint-SI-sample-aligned-corrected.csv", sep="\t", index_col = 0)

df.head(2)

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments,new_translations,substitution_info,substituted_words,errors
0,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg2.1,Začenjam nadaljevanje 12. seje Državnega zbor...,0,7,I am starting a continuation of the 12th meeti...,I am starting a continuation of the 12th meeti...,"{0: 2, 1: 4, 2: 7, 3: 8, 4: 11, 5: 12, 6: 13}",I am starting a continuation of the 12th meeti...,[],[],No
1,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.ana.xml,ParlaMint-SI_2019-10-23-SDZ8-Redna-12.seg3.1,"Obveščen sem , da se današnje seje ne morejo u...","{15: ['Eva', 'Eva'], 16: ['Irgl', 'Irgl'], 19:...",89,I am informed that today's meeting cannot be a...,I am informed that today's meeting cannot be a...,"{0: 2, 1: 1, 3: 3, 5: 4, 6: 5, 8: 6, 9: 8, 10:...",I am informed that today's meeting cannot be a...,"[""No substitution: ('Eva', 'Eva')"", ""No substi...","[('Vrtovac', 'Vrtovec'), ('Serva', 'Sluga'), (...",No
